# Q3

### Importing 

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def get_ind_file(filetype, weighting="vw", n_inds=30):
    """
    Load and format the Ken French Industry Portfolios files
    Variant is a tuple of (weighting, size) where:
        weighting is one of "ew", "vw"
        number of inds is 30 or 49
    """    
    if filetype == "returns":
        name = f"{weighting}_rets" 
        divisor = 100
    elif filetype == "nfirms":
        name = "nfirms"
        divisor = 1
    elif filetype == "size":
        name = "size"
        divisor = 1
    else:
        raise ValueError(f"filetype must be one of: returns, nfirms, size")
    
    ind = pd.read_csv(f"data/ind{n_inds}_m_{name}.csv", header=0, index_col=0, na_values=-99.99)/divisor
    ind.index = pd.to_datetime(ind.index, format="%Y%m").to_period('M')
    ind.columns = ind.columns.str.strip()
    return ind


def get_ind_returns(weighting="vw", n_inds=30):
    """
    Load and format the Ken French Industry Portfolios Monthly Returns
    """
    return get_ind_file("returns", weighting=weighting, n_inds=n_inds)

def get_ind_nfirms(n_inds=30):
    """
    Load and format the Ken French 30 Industry Portfolios Average number of Firms
    """
    return get_ind_file("nfirms", n_inds=n_inds)

def get_ind_size(n_inds=30):
    """
    Load and format the Ken French 30 Industry Portfolios Average size (market cap)
    """
    return get_ind_file("size", n_inds=n_inds)


def get_ind_market_caps(n_inds=30, weights=False):
    """
    Load the industry portfolio data and derive the market caps
    """
    ind_nfirms = get_ind_nfirms(n_inds=n_inds)
    ind_size = get_ind_size(n_inds=n_inds)
    ind_mktcap = ind_nfirms * ind_size
    if weights:
        total_mktcap = ind_mktcap.sum(axis=1)
        ind_capweight = ind_mktcap.divide(total_mktcap, axis="rows")
        return ind_capweight
    #else
    return ind_mktcap

def get_total_market_index_returns(n_inds=30):
    """
    Load the 30 industry portfolio data and derive the returns of a capweighted total market index
    """
    ind_capweight = get_ind_market_caps(n_inds=n_inds)
    ind_return = get_ind_returns(weighting="vw", n_inds=n_inds)
    total_market_return = (ind_capweight * ind_return).sum(axis="columns")
    return total_market_return

In [14]:

ind49_rets = get_ind_returns(weighting="vw", n_inds=49)["2013":]
ind49_mcap = get_ind_market_caps(49, weights=True)["2013":]

In [17]:
columns = ['Hlth','Fin','Whlsl','Rtail','Food']

In [47]:
ind5_rets = ind49_rets[columns]
ind5_mcap = ind49_mcap[columns]


### Market Cap Weights

get the first values of weigths

In [49]:
def weight_cw(r, cap_weights, **kwargs):
    """
    Returns the weights of the CW portfolio based on the time series of capweights
    """
    w = cap_weights.loc[r.index[0]]
    return w/w.sum()

In [92]:
cw=weight_cw(ind5_rets,ind5_mcap)

In [169]:
cw

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
Name: 2013-01, dtype: float64

##### q1 - largest capweight

In [170]:
cw[cw == cw.max()]

Rtail    0.546388
Name: 2013-01, dtype: float64

In [106]:
#hardcode
industries = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
ind_weights = get_ind_market_caps(n_inds=49, weights=True)['2013-01'][industries]

ind_w = pd.Series(ind_weights['2013-01'].values[0], index=industries)

ind_w = ind_w / ind_w.sum()

In [105]:
ind_w

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
dtype: float64

In [58]:
cw[cw==cw.max()]

Rtail    0.546388
Name: 2013-01, dtype: float64

### correlation matrix

In [61]:
rho =ind5_rets.corr()
rho

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,1.000000,0.524886,0.650900,0.574227,0.371698
Fin,0.524886,1.000000,0.777653,0.595332,0.343016
Whlsl,0.650900,0.777653,1.000000,0.743754,0.541244
Rtail,0.574227,0.595332,0.743754,1.000000,0.575063
Food,0.371698,0.343016,0.541244,0.575063,1.000000


### volatility of the assets

In [158]:
vols = ind5_rets.std()*np.sqrt(12)
vols

Hlth     0.162801
Fin      0.167753
Whlsl    0.135644
Rtail    0.141238
Food     0.125401
dtype: float64

In [159]:
vols.dot(vols.T)

0.10871767415492957

### sigma prior ($\Sigma_{prior}$)

In [157]:
sigma_prior = vols.dot(vols.T) * rho
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


$$\pi_{implied} = \delta*\Sigma_{prior}*w$$

In [22]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

In [160]:
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=cw)

In [171]:
pi

Hlth     0.152910
Fin      0.175580
Whlsl    0.201836
Rtail    0.224827
Food     0.158115
Name: Implied Returns, dtype: float64

##### q2 - Highest implied return

In [172]:
pi[pi==pi.max()]

Rtail    0.224827
Name: Implied Returns, dtype: float64

##### q3 0 lowest implied return

In [164]:
pi[pi==pi.min()]

Hlth    0.15291
Name: Implied Returns, dtype: float64

##### Hlth will Out performing by 3% Rtail and Whlsl

In [221]:
cw #weights calculated by first values of weights

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
Name: 2013-01, dtype: float64

In [174]:
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(ind5_rets.columns), index=ind5_rets.columns).T
w_Rtail =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_Whlsl =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


##### q4- p entry for Whlsl

In [173]:
p['Whlsl']

0   -0.151306
Name: Whlsl, dtype: float64

##### q5- p entry for Rtail

In [175]:
p['Rtail']

0   -0.848694
Name: Rtail, dtype: float64

### B&L

In [23]:
def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)

$$\Omega = diag(P (\tau \Sigma) P^T) $$

In [24]:
# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)


In [109]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

In [180]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)

In [182]:
bl_mu

Hlth     0.179643
Fin      0.169252
Whlsl    0.193340
Rtail    0.199847
Food     0.145319
dtype: float64

In [183]:
bl_sigma

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.113542,0.060062,0.074497,0.066121,0.042723
Fin,0.060062,0.114119,0.088726,0.067824,0.039087
Whlsl,0.074497,0.088726,0.114092,0.084721,0.061692
Rtail,0.066121,0.067824,0.084721,0.113620,0.065372
Food,0.042723,0.039087,0.061692,0.065372,0.114013


In [127]:
bl_sigma

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.113542,0.060062,0.074497,0.066121,0.042723
Fin,0.060062,0.114119,0.088726,0.067824,0.039087
Whlsl,0.074497,0.088726,0.114092,0.084721,0.061692
Rtail,0.066121,0.067824,0.084721,0.113620,0.065372
Food,0.042723,0.039087,0.061692,0.065372,0.114013


In [128]:
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

In [222]:
def w_star(delta,sigma,mu):
    return (inverse(sigma).dot(mu))/delta

###### weights returned by bl

In [154]:
w_star_01 = pd.Series(w_star(delta=2.5, sigma = bl_sigma, mu = bl_mu))

In [155]:
w_star_01

Hlth     0.263700
Fin      0.167011
Whlsl    0.058876
Rtail    0.330244
Food     0.132549
dtype: float64

In [225]:
pi_w3= implied_returns(delta=2.5, sigma=sigma_prior, w=w_star_01)
pi_w3

Hlth     0.170847
Fin      0.161250
Whlsl    0.184211
Rtail    0.190557
Food     0.138515
Name: Implied Returns, dtype: float64

##### q6 - Lowest implied return

In [224]:
pi_w3[pi_w3 == pi_w3.min()]

Food    0.138515
Name: Implied Returns, dtype: float64

In [195]:
def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [230]:
w_eq  = w_msr(delta*sigma_prior, pi, scale=False)

In [232]:
w_eq

Hlth     0.041663
Fin      0.175362
Whlsl    0.097411
Rtail    0.546388
Food     0.139176
dtype: float64

In [ ]:
w_

In [ ]:
w_bl = w_msr(delta*)

##### q7 - Highest MSR

In [208]:
w_eq[w_eq == w_eq.max()]

Rtail    0.546388
dtype: float64

##### q8 - Lowest MSR

In [210]:
w_eq[w_eq == w_eq.min()]

Hlth    0.041663
dtype: float64

##### Hlth will Out performing by 5% Rtail and Whlsl

In [211]:
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(ind5_rets.columns), index=ind5_rets.columns).T
w_Rtail =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_Whlsl =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


In [212]:
delta = 2.5
tau = 0.05
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)

In [215]:
bl_mu[bl_mu == bl_mu.max()]

Rtail    0.194772
dtype: float64

In [219]:
wstar_02 = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar_02

Hlth     0.310774
Fin      0.167011
Whlsl    0.051754
Rtail    0.290293
Food     0.132549
dtype: float64